In [ ]:
import statsmodels.api as sm
import random
import numpy as np
import pandas as pd

import scipy
import scipy.linalg as slin
from scipy.stats import t

In [ ]:
##data was calculated in R  and written in  test_df.csv

##set.seed(42)
##x <- sin(1:100)
##y <- rnorm(100)

np.random.seed(42)
df = pd.read_csv('test_df.csv', sep = ',', index_col= 0)
x = pd.DataFrame([df.x, df.x**2]).T
y = df.y
x = sm.add_constant(x)
model = sm.OLS(y, x)
model.fit().summary()

In [ ]:
ell = [1, 0, np.repeat(0,model.rank-2)[0]]
clustervar = [np.repeat(1, 40), np.repeat(1, 20), np.repeat(2, 20), np.repeat(3, 10), np.repeat(4, 10)]
clust = [item for sublist in clustervar for item in sublist]

def dfadjustSE(model, clustervar='NULL', ell='NULL', IK='TRUE', tol=1e-9,
                       rho0='FALSE'):
    R = np.linalg.qr(model.exog, mode = 'r')
    Q = np.linalg.qr(model.exog)[0]
    n = len(Q)
    u = model.fit().resid
    K = model.rank
    rho = sig = 'NA'
    
    def function(HC2):
        return slin.solve_triangular(R, (slin.solve_triangular(R, HC2,lower=False)).T)

    if (clustervar == 'NULL'):
        influence = model.fit().get_influence()
        diaghat = influence.hat_matrix_diag

        chet = (1-diaghat >= tol) * 1/np.sqrt(np.maximum(1-diaghat, tol))  
        chet1 = np.tile(chet,(3,1)).T
        AQ = chet1 * Q
        u1 = np.tile(u,(3,1)).T
        X = AQ * u1
        X1 = Q * u1
        HC2 = np.dot(X.T, X)
        HC1 = n/(n-K)*np.dot(X1.T, X1)
        
        def func(ell):
            a = np.dot(AQ, slin.solve_triangular(R, ell, trans = 1))
            a1 = np.tile(a,(3,1)).T
            B = a1*Q
            return (np.sum(a**2)-np.sum(B**2))**2 / (np.sum(a**4)-2*np.sum((a1*B)**2)+np.sum(np.dot(B.T, B)**2))

    else:
        clust = [item for sublist in clustervar for item in sublist]
        S = len(set(clust))
        X11 = pd.DataFrame(Q * np.tile(u,(3,1)).T)
        X11['clust'] = clust
        uj = X11.groupby('clust').sum()
        HC1 = S/(S-1) * (n-1)/(n-K) * np.dot(uj.T, uj)
        def AQf(s):
            Q11 = pd.DataFrame(Q)
            Q11['clust'] = clust
            Qs = Q11[Q11['clust']==s] 
            Qs = Qs.iloc[:,:-1]
            e = np.linalg.eig(np.dot(Qs.T, Qs))
            Ds = e[1] @ ((1-e[0] >= tol) *(1/np.sqrt(np.maximum(1-e[0], tol))) * e[1]).T
            return np.dot(Qs, Ds)
    
        AQ = []
        for i in range(1,S+1):
            AQ.append(list(AQf(i)))
        AQ = pd.DataFrame([item for sublist in AQ for item in sublist])
        dfAQ = AQ * np.tile(u,(3,1)).T
        dfAQ['clust'] = clust
        uj = dfAQ.groupby('clust').sum()
        HC2 = np.dot(uj.T,uj)
        
        if (IK == 'TRUE'):
            ssr = np.sum(u**2)
            den = np.sum(pd.DataFrame([[x,clust.count(x)] for x in set(clust)]).iloc[:,1]**2) - n
            rho = 0 
            if (den>0):
                df_u = pd.DataFrame(u)
                df_u['clust'] = clust
                us = df_u.groupby('clust').sum()**2
                rho= (np.sum(us)-ssr) / den
            if (rho0 == 'TRUE'):
                rho[0] = np.max([rho[0], 0])
                
            try:
                sig = max(ssr/n - rho, 0)
            except:
                sig = max(ssr/n - rho[0], 0)
                
        def func(ell):
            a = np.dot(AQ, slin.solve_triangular(R, ell, trans = 1))
            a_df = pd.DataFrame(a**2)
            a_df['clust'] = clust
            a_s = a_df.groupby(clust).sum()[0]
            a_s1 = np.tile(a_s,(3,1)).T
            
            a11 = np.tile(a,(3,1)).T
            aQ = pd.DataFrame(a11 * Q)
            aQ['clust'] = clust
            B = aQ.groupby(clust).sum().iloc[:,:-1]
            if (IK == 'False'):
                return (np.sum(a_s)-np.sum(np.sum(B**2)))**2 / (np.sum(a_s**2)-2*np.sum(np.sum(a_s1*B**2))+np.sum(np.dot(B.T, B)**2))
            else:
                a_df = pd.DataFrame(a)
                a_df['clust'] = clust
                D = a_df.groupby(clust).sum()[0] 
                Q_df = pd.DataFrame(Q)
                Q_df['clust'] = clust
                Fm = Q_df.groupby(clust).sum().iloc[:,:-1]
                
                GG = sig*(np.diag(a_s) - B @ B.T) + rho[0] * ((np.diag(D) - B @ Fm.T) @ (np.diag(D) - B @ Fm.T).T)
                
    
                return np.sum(np.diag(GG))**2 / np.sum(np.sum(GG**2))

    def function(HC2):
        return slin.solve_triangular(R, (slin.solve_triangular(R, HC2,lower=False)).T)
    Vhat = function(HC2)
    VhatStata = function(HC1)
    

    if (ell != 'NULL'):
        se =np.sqrt(ell @ Vhat @ ell)
        dof = func(ell)
        seStata = np.sqrt(np.dot(ell,VhatStata) @ ell)
        beta = np.sum(ell*model.fit().params)
    
    else:
        se = np.sqrt(np.diag(Vhat))
        dof = [func(np.eye(K)[:, k]) for k in range(K)]
        seStata = np.sqrt(np.diag(VhatStata))
        beta = model.fit().params
    if (ell == 'NULL'):
        df = pd.DataFrame({"Estimate":beta,
             "HC1 se":seStata,
             "HC2 se":se,
             "Adj. se":se*scipy.stats.t.ppf(0.975, df=dof)/scipy.stats.norm.ppf(0.975),
             "df":dof,
             "p-value": 2*scipy.stats.t.cdf(-np.abs(model.fit().params/se), df = dof)})
    else:
        p_val = 2*scipy.stats.t.cdf(-np.abs(model.fit().params/se), df = dof) * ell 
        df = pd.DataFrame({"Estimate":beta,
             "HC1 se":seStata,
             "HC2 se":se,
             "Adj. se":se*scipy.stats.t.ppf(0.975, df=dof)/scipy.stats.norm.ppf(0.975),
             "df":dof,
             "p-value": p_val})
        df = df[df['p-value']!=0]

    return df

dfadjustSE(model, IK = 'False', ell = ell)